In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from contextlib import closing
from selenium.webdriver import Firefox # pip install selenium
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from time import sleep

import pysentiment2 as ps

# New York Times

In [2]:

url = 'https://www.nytimes.com/search?dropmab=true&endDate=20200423&query=Trump&sort=oldest&startDate=20200318&types=article'
url = 'https://www.nytimes.com/search?dropmab=false&endDate=20200520&query=Trump&sort=oldest&startDate=20200423&types=article'

#url = 'https://www.dailymail.co.uk'

In [3]:
# Makes a HTTP request to the website
r1 = requests.get(url)
r1.status_code

# Returns the unicode content of the request response
coverpage = r1.content

# Soup creation
soup1 = BeautifulSoup(coverpage, 'html5lib')

# News identification
# coverpage_news = soup1.find_all('h2', class_='linkro-darkred')
# len(coverpage_news)

In [4]:
# Scraping the first 5 articles
number_of_articles = 5# Empty lists for content, links and titles
news_contents = []
list_links = []
list_titles = []

for n in np.arange(0, number_of_articles):
    
    # only news articles (there are also albums and other things)
    if "inenglish" not in coverpage_news[n].find('a')['href']:  
        continue
    
    # Getting the link of the article
    link = coverpage_news[n].find('a')['href']
    list_links.append(link)
    
    # Getting the title
    title = coverpage_news[n].find('a').get_text()
    list_titles.append(title)
    
    # Reading the content (it is divided in paragraphs)
    article = requests.get(link)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')
    body = soup_article.find_all('div', class_='articulo-cuerpo')
    x = body[0].find_all('p')
    
    # Unifying the paragraphs
    list_paragraphs = []
    for p in np.arange(0, len(x)):
        paragraph = x[p].get_text()
        list_paragraphs.append(paragraph)
        final_article = " ".join(list_paragraphs)
        
    news_contents.append(final_article)

NameError: name 'coverpage_news' is not defined

In [5]:
soup1.find('button', text='Show More').attrs

{'data-testid': 'search-show-more-button', 'type': 'button'}

In [6]:
all_the_text = []

In [7]:
all_links = soup1.find_all('a')
articles = []
for a in all_links:
    if a.find('h4'):
        articles.append(a)

In [125]:
link = 'http://www.nytimes.com'+articles[0]['href']

In [126]:
article = requests.get(link)
article_content = article.content
soup_article = BeautifulSoup(article_content, 'html5lib')
body = soup_article.find_all('div', class_='articulo-cuerpo')

In [127]:
paragraphs = soup_article.find_all('p')[4:-2]

In [128]:
body = ''
for i in range(len(paragraphs)):
    body += ' ' + paragraphs[i].get_text()

In [178]:
analyzer = SentimentIntensityAnalyzer()

In [134]:
analyzer.polarity_scores(body)

{'neg': 0.077, 'neu': 0.858, 'pos': 0.065, 'compound': -0.9174}

In [9]:
#Period 1
url = 'https://www.nytimes.com/search?dropmab=true&endDate=20200423&query=Trump&sort=oldest&startDate=20200318&types=article'

with closing(Firefox()) as driver:
    driver.get(url)
    soup = BeautifulSoup(driver.page_source)
    while True:#soup.find('button', text='Show More') is not None:
        try:
            button = driver.find_element(By.XPATH, '//button[text()="Show More"]')
        except:
            break
        button.click()
        WebDriverWait(driver, 10)
        # store it to string variable
        soup = BeautifulSoup(driver.page_source)
    page_source = driver.page_source
print("Pressed the Show More button a bunch")    


del soup
soup1 = BeautifulSoup(page_source, 'html5lib')
all_links = soup1.find_all('a')
articles = []
for a in all_links:
    if a.find('h4'):
        articles.append(a)
print("Got all the Articles")

#all_the_text = []
scores = []
#analyzer = SentimentIntensityAnalyzer()
hiv4 = ps.HIV4()

print('Scraping...')
print('\r[' + '>' + '-'*19 + ']', end=' ', flush=True)

for i in range(len(articles)):   
    link = 'http://www.nytimes.com'+articles[i]['href']
    #print("Scraping ", link)
    article = requests.get(link)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')
    paragraphs = soup_article.find_all('p')[4:-2]
    body = ''
    for j in range(len(paragraphs)):
        body += ' ' + paragraphs[j].get_text()
    #all_the_text.append(body)
    #if 'Covid' in body or 'Coronavirus' in body:
       # scores.append(analyzer.polarity_scores(body))
    tokens = hiv4.tokenize(body)
    scores.append(hiv4.get_score(tokens))
    temp = int(i // (len(articles)/20))
    if temp == 0: temp = 1
    print('\r[' + '='*(temp-1) + '>' + '-'*(20-temp) + ']', end=' ', flush=True)
print('\r[' + '='*(19) + '>' + ']', end=' ', flush=True)
print("Analyzed the articles")

Pressed the Show More button a bunch
Got all the Articles
Scraping...
[==============>-----] 

KeyboardInterrupt: 

In [10]:
#836 articles
len(scores)
#Compund 0.04011555023923443

670

In [11]:
av = 0
for score in scores:
    av += score['Polarity']
av /= len(scores)
av

0.1834964322443599

In [224]:
#Period 2
url = 'https://www.nytimes.com/search?dropmab=false&endDate=20200520&query=Trump&sort=oldest&startDate=20200423&types=article'
with closing(Firefox()) as driver:
    driver.get(url)
    soup = BeautifulSoup(driver.page_source)
    while True:#soup.find('button', text='Show More') is not None:
        try:
            button = driver.find_element(By.XPATH, '//button[text()="Show More"]')
        except:
            break
        button.click()
        WebDriverWait(driver, 10)
        # store it to string variable
        soup = BeautifulSoup(driver.page_source)
    page_source = driver.page_source
print("Pressed the Show More button a bunch")    


del soup
soup1 = BeautifulSoup(page_source, 'html5lib')
all_links = soup1.find_all('a')
articles = []
for a in all_links:
    if a.find('h4'):
        articles.append(a)
print("Got all the Articles")

#all_the_text = []
scores2 = []
print('Scraping...')
print('\r[' + '>' + '-'*19 + ']', end=' ', flush=True)

scores2 = []
#analyzer = SentimentIntensityAnalyzer()
hiv4 = ps.HIV4()

print('Scraping...')
print('\r[' + '>' + '-'*19 + ']', end=' ', flush=True)

for i in range(len(articles)):   
    link = 'http://www.nytimes.com'+articles[i]['href']
    #print("Scraping ", link)
    article = requests.get(link)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')
    paragraphs = soup_article.find_all('p')[4:-2]
    body = ''
    for j in range(len(paragraphs)):
        body += ' ' + paragraphs[j].get_text()
    #all_the_text.append(body)
    #if 'Covid' in body or 'Coronavirus' in body:
       # scores.append(analyzer.polarity_scores(body))
    tokens = hiv4.tokenize(body)
    scores2.append(hiv4.get_score(tokens))
    temp = int(i // (len(articles)/20))
    if temp == 0: temp = 1
    print('\r[' + '='*(temp-1) + '>' + '-'*(20-temp) + ']', end=' ', flush=True)
print('\r[' + '='*(19) + '>' + ']', end=' ', flush=True)
print("Analyzed the articles")

Pressed the Show More button a bunch
Got all the Articles
Scraping...
[>-------------------] Scraping...
[===================>] Analyzed the articles


In [225]:
#642 articles

len(scores2)
#Compund 0.21758286604361396

692

In [227]:
av2 = 0
for score in scores2:
    av2 += score['Polarity']
av2 /= len(scores2)
av2

0.17382998320361828

# Fox News

In [233]:
#XPath to date-range
url = 'https://www.foxnews.com/search-results/search?q=Trump'
with closing(Firefox()) as driver:
    driver.get(url)
    
    #choose the article content type
    content = driver.find_element_by_xpath("//div[@class='filter content']/button")
    content.click()
    content_article = driver.find_element_by_xpath("//input[@value='Article']")
    content_article.click()
    
    #we have to manually choose the date range for our search
    min_month = driver.find_element_by_xpath("//div[@class='date min']/div[@class='sub month']")
    min_month.click()
    min_month_choice = driver.find_element_by_xpath("//div[@class='date min']/div[@class='sub month']/ul[@class='option']/li[@class='03']")
    min_month_choice.click()
    
    min_day = driver.find_element_by_xpath("//div[@class='date min']/div[@class='sub day']")
    min_day.click()
    min_day_choice = driver.find_element_by_xpath("//div[@class='date min']/div[@class='sub day']/ul[@class='option']/li[@class='18']")
    min_day_choice.click()
    
    min_year = driver.find_element_by_xpath("//div[@class='date min']/div[@class='sub year']")
    min_year.click()
    min_year_choice = driver.find_element_by_xpath("//div[@class='date min']/div[@class='sub year']/ul[@class='option']/li[@id='2020']")
    min_year_choice.click()
    
    
    max_month = driver.find_element_by_xpath("//div[@class='date max']/div[@class='sub month']")
    max_month.click()
    max_month_choice = driver.find_element_by_xpath("//div[@class='date max']/div[@class='sub month']/ul[@class='option']/li[@class='04']")
    max_month_choice.click()
    
    max_day = driver.find_element_by_xpath("//div[@class='date max']/div[@class='sub day']")
    max_day.click()
    max_day_choice = driver.find_element_by_xpath("//div[@class='date max']/div[@class='sub day']/ul[@class='option']/li[@class='22']")
    max_day_choice.click()
    
    max_year = driver.find_element_by_xpath("//div[@class='date max']/div[@class='sub year']")
    max_year.click()
    max_year_choice = driver.find_element_by_xpath("//div[@class='date max']/div[@class='sub year']/ul[@class='option']/li[@id='2020']")
    max_year_choice.click()
    
    search_button = driver.find_element_by_xpath("//a[text()='Search']")#"//div[@class='search-form']/div[@class='button']")#
    search_button.click()
    search_button.click()
    WebDriverWait(driver, 10000)
    
    #now hit the load more button as much as possible
    while True:
        try:
            button = driver.find_element_by_class_name('load-more')
            button.click()
            WebDriverWait(driver, 100)
        except:
            break
    page_source = driver.page_source

soup = BeautifulSoup(page_source, 'html5lib')
all_articles = soup.find_all('article')
links = []
for a in all_articles:
    links.append(a.find('a')['href'])
print("Got all the Articles")

scores = []
#analyzer = SentimentIntensityAnalyzer()
hiv4 = ps.HIV4()

print('Scraping...')
print('\r[' + '>' + '-'*19 + ']', end=' ', flush=True)

for i, link in enumerate(links):   
    article = requests.get(link)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')
    paragraphs = soup_article.find_all('p')[3:-6]
    body = ''
    for j in range(len(paragraphs)):
        body += ' ' + paragraphs[j].get_text()

    body = body.replace('\xa0', ' ')
    #if 'Covid' in body or 'Coronavirus' in body:
    #scores.append(analyzer.polarity_scores(body))
    tokens = hiv4.tokenize(body)
    scores.append(hiv4.get_score(tokens))
    temp = int(i // (len(links)/20))
    if temp == 0: temp = 1
    print('\r[' + '='*(temp-1) + '>' + '-'*(20-temp) + ']', end=' ', flush=True)
print('\r[' + '='*(19) + '>' + ']', end=' ', flush=True)
print("Analyzed the articles")

Got all the Articles
Scraping...
[===================>] Analyzed the articles


In [234]:
#642 articles

len(scores)
#Compund 10.833333333333334

55

In [235]:
av2 = 0
for score in scores:
    av2 += score['Polarity']
av2 /= len(scores)
av2

0.10756928274876155

In [2]:
#XPath to date-range
url = 'https://www.foxnews.com/search-results/search?q=Trump'
with closing(Firefox()) as driver:
    driver.get(url)
    
    #choose the article content type
    content = driver.find_element_by_xpath("//div[@class='filter content']/button")
    content.click()
    content_article = driver.find_element_by_xpath("//input[@value='Article']")
    content_article.click()
    
    #we have to manually choose the date range for our search
    min_month = driver.find_element_by_xpath("//div[@class='date min']/div[@class='sub month']")
    min_month.click()
    min_month_choice = driver.find_element_by_xpath("//div[@class='date min']/div[@class='sub month']/ul[@class='option']/li[@class='04']")
    min_month_choice.click()
    
    min_day = driver.find_element_by_xpath("//div[@class='date min']/div[@class='sub day']")
    min_day.click()
    min_day_choice = driver.find_element_by_xpath("//div[@class='date min']/div[@class='sub day']/ul[@class='option']/li[@class='22']")
    min_day_choice.click()
    
    min_year = driver.find_element_by_xpath("//div[@class='date min']/div[@class='sub year']")
    min_year.click()
    min_year_choice = driver.find_element_by_xpath("//div[@class='date min']/div[@class='sub year']/ul[@class='option']/li[@id='2020']")
    min_year_choice.click()
    
    
    max_month = driver.find_element_by_xpath("//div[@class='date max']/div[@class='sub month']")
    max_month.click()
    max_month_choice = driver.find_element_by_xpath("//div[@class='date max']/div[@class='sub month']/ul[@class='option']/li[@class='05']")
    max_month_choice.click()
    
    max_day = driver.find_element_by_xpath("//div[@class='date max']/div[@class='sub day']")
    max_day.click()
    max_day_choice = driver.find_element_by_xpath("//div[@class='date max']/div[@class='sub day']/ul[@class='option']/li[@class='20']")
    max_day_choice.click()
    
    max_year = driver.find_element_by_xpath("//div[@class='date max']/div[@class='sub year']")
    max_year.click()
    max_year_choice = driver.find_element_by_xpath("//div[@class='date max']/div[@class='sub year']/ul[@class='option']/li[@id='2020']")
    max_year_choice.click()
    
    search_button = driver.find_element_by_xpath("//a[text()='Search']")#"//div[@class='search-form']/div[@class='button']")#
    search_button.click()
    search_button.click()
    WebDriverWait(driver, 10000)
    
    #now hit the load more button as much as possible
    while True:
        try:
            button = driver.find_element_by_class_name('load-more')
            button.click()
            WebDriverWait(driver, 100)
        except:
            break
    page_source = driver.page_source

soup = BeautifulSoup(page_source, 'html5lib')
all_articles = soup.find_all('article')
links = []
for a in all_articles:
    links.append(a.find('a')['href'])
print("Got all the Articles")

scores = []
#analyzer = SentimentIntensityAnalyzer()
hiv4 = ps.HIV4()

print('Scraping...')
print('\r[' + '>' + '-'*19 + ']', end=' ', flush=True)

for i, link in enumerate(links):   
    article = requests.get(link)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')
    paragraphs = soup_article.find_all('p')[3:-6]
    body = ''
    for j in range(len(paragraphs)):
        body += ' ' + paragraphs[j].get_text()

    body = body.replace('\xa0', ' ')
    #if 'Covid' in body or 'Coronavirus' in body:
    #scores.append(analyzer.polarity_scores(body))
    tokens = hiv4.tokenize(body)
    scores.append(hiv4.get_score(tokens))
    temp = int(i // (len(links)/20))
    if temp == 0: temp = 1
    print('\r[' + '='*(temp-1) + '>' + '-'*(20-temp) + ']', end=' ', flush=True)
print('\r[' + '='*(19) + '>' + ']', end=' ', flush=True)
print("Analyzed the articles")

KeyboardInterrupt: 

In [237]:
#642 articles

len(scores)
#Compund 10.833333333333334

37

In [238]:
av2 = 0
for score in scores:
    av2 += score['Polarity']
av2 /= len(scores)
av2

0.15725378004509794

# CNN

In [ ]:
#XPath to date-range
url = 'https://www.cnn.com/search?q=Trump&sort=relevance&type=article&size=30&page=1'
with closing(Firefox()) as driver:
    driver.get(url)
    
    #choose the article content type
    content = driver.find_element_by_xpath("//div[@class='filter content']/button")
    content.click()
    content_article = driver.find_element_by_xpath("//input[@value='Article']")
    content_article.click()
    
    #we have to manually choose the date range for our search
    min_month = driver.find_element_by_xpath("//div[@class='date min']/div[@class='sub month']")
    min_month.click()
    min_month_choice = driver.find_element_by_xpath("//div[@class='date min']/div[@class='sub month']/ul[@class='option']/li[@class='04']")
    min_month_choice.click()
    
    min_day = driver.find_element_by_xpath("//div[@class='date min']/div[@class='sub day']")
    min_day.click()
    min_day_choice = driver.find_element_by_xpath("//div[@class='date min']/div[@class='sub day']/ul[@class='option']/li[@class='22']")
    min_day_choice.click()
    
    min_year = driver.find_element_by_xpath("//div[@class='date min']/div[@class='sub year']")
    min_year.click()
    min_year_choice = driver.find_element_by_xpath("//div[@class='date min']/div[@class='sub year']/ul[@class='option']/li[@id='2020']")
    min_year_choice.click()
    
    
    max_month = driver.find_element_by_xpath("//div[@class='date max']/div[@class='sub month']")
    max_month.click()
    max_month_choice = driver.find_element_by_xpath("//div[@class='date max']/div[@class='sub month']/ul[@class='option']/li[@class='05']")
    max_month_choice.click()
    
    max_day = driver.find_element_by_xpath("//div[@class='date max']/div[@class='sub day']")
    max_day.click()
    max_day_choice = driver.find_element_by_xpath("//div[@class='date max']/div[@class='sub day']/ul[@class='option']/li[@class='20']")
    max_day_choice.click()
    
    max_year = driver.find_element_by_xpath("//div[@class='date max']/div[@class='sub year']")
    max_year.click()
    max_year_choice = driver.find_element_by_xpath("//div[@class='date max']/div[@class='sub year']/ul[@class='option']/li[@id='2020']")
    max_year_choice.click()
    
    search_button = driver.find_element_by_xpath("//a[text()='Search']")#"//div[@class='search-form']/div[@class='button']")#
    search_button.click()
    search_button.click()
    WebDriverWait(driver, 10000)
    
    #now hit the load more button as much as possible
    while True:
        try:
            button = driver.find_element_by_class_name('load-more')
            button.click()
            WebDriverWait(driver, 100)
        except:
            break
    page_source = driver.page_source

soup = BeautifulSoup(page_source, 'html5lib')
all_articles = soup.find_all('article')
links = []
for a in all_articles:
    links.append(a.find('a')['href'])
print("Got all the Articles")

scores = []
#analyzer = SentimentIntensityAnalyzer()
hiv4 = ps.HIV4()

print('Scraping...')
print('\r[' + '>' + '-'*19 + ']', end=' ', flush=True)

for i, link in enumerate(links):   
    article = requests.get(link)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')
    paragraphs = soup_article.find_all('p')[3:-6]
    body = ''
    for j in range(len(paragraphs)):
        body += ' ' + paragraphs[j].get_text()

    body = body.replace('\xa0', ' ')
    #if 'Covid' in body or 'Coronavirus' in body:
    #scores.append(analyzer.polarity_scores(body))
    tokens = hiv4.tokenize(body)
    scores.append(hiv4.get_score(tokens))
    temp = int(i // (len(links)/20))
    if temp == 0: temp = 1
    print('\r[' + '='*(temp-1) + '>' + '-'*(20-temp) + ']', end=' ', flush=True)
print('\r[' + '='*(19) + '>' + ']', end=' ', flush=True)
print("Analyzed the articles")

In [8]:
url = 'https://www.cnn.com/search?q=Trump&sort=date&type=article&size=30&from=0&page=1'
# start_date = pd.to_datetime('03/18/2020')
# end_date = pd.to_datetime('04/22/2020')
start_date = pd.to_datetime('04/22/2020')
end_date = pd.to_datetime('5/20/20')

body = []
with closing(Firefox()) as driver:
    driver.get(url)
    
    can_continue = True
    current_page = 1
    while can_continue:
        #choose the article content type

        
        for i in range(1, 31):
            try:
                content = driver.find_element_by_xpath(f"//div[@class='cnn-search__results-list']/div[{i}]")
                date = pd.to_datetime(content.text.split('\n')[1])
                if i == 1:
                    print(content.text.split('\n')[0])
                if start_date < date < end_date:
                    body.append(content.text)
            except:
                WebDriverWait(driver, 1000)
                continue
            
        
        try:
            next_arrow = driver.find_element_by_xpath("//div[@class='pagination-arrow pagination-arrow-right cnnSearchPageLink text-active']")
            next_arrow.click()
#             current_page += 1
#             url = f'https://www.cnn.com/search?q=Trump&sort=relevance&type=article&size=30&from={30*(current_page-1)}&page={current_page}'
#             driver.get(url)
            WebDriverWait(driver, 1500)
        except:
            break


Trump: Europe is terrific place for investment
Trump: Europe is terrific place for investment
Ghostwriter: 'The inner Trump is the outer Trump'
Ghostwriter: 'The inner Trump is the outer Trump'
Ghostwriter: 'The inner Trump is the outer Trump'
Ghostwriter: 'The inner Trump is the outer Trump'
Ghostwriter: 'The inner Trump is the outer Trump'
Ghostwriter: 'The inner Trump is the outer Trump'
Paul Ryan endorses Donald Trump
Paul Ryan endorses Donald Trump
Paul Ryan endorses Donald Trump
Trump brand too hot to handle
Trump brand too hot to handle
Trump brand too hot to handle
Trump brand too hot to handle
Donald Trump: 'Radical Muslims' a problem
Donald Trump: 'Radical Muslims' a problem
Donald Trump: 'Radical Muslims' a problem
Donald Trump: 'Radical Muslims' a problem
Donald Trump thinks you're stupid
Donald Trump thinks you're stupid
Donald Trump thinks you're stupid
Donald Trump thinks you're stupid
Trump is impeaching himself
Trump is impeaching himself
Trump is impeaching himself
Tr

In [10]:
len(body)

35

In [15]:
for b in body:
    print(b.split('\n')[1])
    

May 13, 2020
May 13, 2020
May 13, 2020
May 13, 2020
May 13, 2020
May 13, 2020
Apr 25, 2020
Apr 25, 2020
May 4, 2020
May 4, 2020
May 4, 2020
May 4, 2020
May 4, 2020
May 4, 2020
Apr 27, 2020
Apr 27, 2020
Apr 27, 2020
May 18, 2020
May 18, 2020
May 18, 2020
May 18, 2020
May 9, 2020
May 9, 2020
May 9, 2020
May 18, 2020
May 18, 2020
May 18, 2020
May 18, 2020
May 2, 2020
May 2, 2020
May 12, 2020
May 12, 2020
May 12, 2020
May 18, 2020
May 18, 2020
